In [10]:
%pip install gensim
%pip install torch
%pip install Pillow
%pip install requests
%pip install open_clip_torch
%pip install torchvision


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [11]:
import torch
import torch.nn as nn
import torch.optim as optim
import gensim.downloader as api # pip install gensim
from txt2png import txt_to_png
from open_clip_test_mps import clip_loss, device, preprocess

In [12]:

word_model_name="glove-twitter-25"
word_model = api.load(word_model_name)

embedding_dim = 25

output_size = 15 * 6


ascii_chars = " .,:;+*#@$%&0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ-_=|/\\()[]{}"
num_chars = len(ascii_chars)
char_to_index = {char: i for i, char in enumerate(ascii_chars)}
index_to_char = {i: char for i, char in enumerate(ascii_chars)}

class ASCIIArtGenerator(nn.Module):
    def __init__(self, input_size, output_size, num_chars):
        super(ASCIIArtGenerator, self).__init__()
        self.fc1 = nn.Linear(input_size, 512)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, output_size * num_chars)
        self.num_chars = num_chars
        self.output_size = output_size

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x.view(-1, self.output_size, self.num_chars) 

def generate_ascii_art(model, embedding):
    model.eval()
    with torch.no_grad():
        output = model(embedding.unsqueeze(0))
        _, indices = torch.max(output, dim=2)
        ascii_grid = ""
        for i in range(6):
            for j in range(15):
                ascii_grid += index_to_char[indices[0][i * 15 + j].item()]
            ascii_grid += "\n"
    return ascii_grid


model = ASCIIArtGenerator(embedding_dim, output_size, num_chars)

In [14]:
embedding = torch.tensor(word_model["cat"])
for i in range(10):
    optimizer = optim.Adam(model.parameters(), lr = .001)
    ascii_art = generate_ascii_art(model, embedding)
    ascii_png = txt_to_png(ascii_art, output_png=None)
    #image = Image.open("./Data/Image/castle_710.png")
    image_tensor = preprocess(ascii_png).unsqueeze(0).to(device, dtype=torch.float32)
    optimizer.zero_grad()
    #generated_images = generator(noise)          # Your generator produces images.
    candidate_labels = ["a bird", "a dog", "a cat", "a castle"]
    true_label = 2
    loss = clip_loss(ascii_art, candidate_labels, true_label)
    loss.backward()
    optimizer.step()
    print(ascii_art)

FileNotFoundError: [Errno 2] No such file or directory: 'qXd9Z|e=@;c2||Q\n{3*2Aa-Uw1gRGEL\n$wU.Pr\\S[q\\U;y8\ns*Z)zp3{x9]VS2s\n]ukyxDIB}Z&A0#2\nPMT-.{Pzc,nB*U|\n'